# Exploratory Data Analysis para datos de Precios


## Importando Librerías

In [81]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
import io
import os
from dotenv import load_dotenv, find_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import datetime as dt

In [58]:
pd.options.display.max_columns = 100

In [59]:
load_dotenv(find_dotenv())

True

## Leyendo Datos

In [60]:
def obtain_content_of_blob(connection_string, container_name, file_name):
    """
    Esta función crea una conexión a un archivo específico almacenado en blob Storage y retorna su contenido
    """
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(file_name)
    blob_stream = blob_client.download_blob().readall()
    blob_file = io.BytesIO(blob_stream)
    return blob_file

In [61]:
connection_string = os.getenv("AZ_CONNECTION_STRING")
container_name = "raw/EnergySuiteData/precios/aire/"

In [62]:
df_precios = pd.read_csv(
    obtain_content_of_blob(connection_string, container_name, "AireSup-Prices.csv"),
    delimiter=";",
    # type={"Id":str,"Date": str,"Period":int,"Version":int,"ProductType":str,
    #   "ConceptId":str,"ElementId":str,"Value":float,"SourceType":str,"NatureType":str},
    #   nrows=100000) #Loading only first 100000 rows
)
print(df_precios.shape)
print(df_precios.dtypes)
df_precios.head()

(21264, 9)
Id              object
Date            object
Period           int64
Version          int64
MarketType      object
ProductType     object
ConceptId       object
Value          float64
UpLoadId       float64
dtype: object


,Id,Date,Period,Version,MarketType,ProductType,ConceptId,Value,UpLoadId
0,7459BD0B-E48E-487A-B7D9-01E11783FAF0,2023-09-30 00:00:00.000,20,4,TMerEner,TPEner,PBNA,1062.184761,NaN
1,25E1062D-603C-432C-BAA5-07C4013A78FE,2023-09-30 00:00:00.000,19,4,TMerEner,TPEner,PBNA,1062.185761,NaN
2,BABD0C9A-111E-4D7A-A4B7-0AB310F6D4A5,2023-09-30 00:00:00.000,8,4,TMerEner,TPEner,PBNA,1008.944761,NaN
3,5109AA0B-B546-408C-B9A5-1C0FF634D91A,2023-09-30 00:00:00.000,2,4,TMerEner,TPEner,PBNA,1008.945761,NaN
4,F7CCDBA3-1710-4BF7-8BEE-28826207E2FA,2023-09-30 00:00:00.000,11,4,TMerEner,TPEner,PBNA,1062.085761,NaN


## Preprocesando los datos

### Analizando contenido y relevancia de las columnas

In [63]:
df_precios.sort_values(by="Period",ascending=True).Period.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [64]:
df_precios.Version.unique()

array([4, 5, 6, 3])

In [65]:
df_precios.MarketType.unique()

array(['TMerEner'], dtype=object)

In [10]:
df_precios.Period.unique()

array([20, 19,  8,  2, 11, 17, 13, 24, 22, 15,  3, 10,  5,  6, 23, 12, 21,
        4, 16, 14,  9,  1,  7, 18,  0])

In [67]:
df_precios.ProductType.unique()

array(['TPEner'], dtype=object)

In [68]:
df_precios.ConceptId.unique()

array(['PBNA', 'DSNTF', 'DSNTM', 'INDIPC', 'INDIPP', 'MC'], dtype=object)

In [69]:
df_precios.groupby(["ConceptId"])[["ConceptId"]].count()

,ConceptId
ConceptId,
DSNTF,1690
DSNTM,191
INDIPC,14
INDIPP,15
MC,14
PBNA,19340


Se identifican diferentes Conceptos para el valor de la energía, según la regla de negocio indicada, solo se deja el concepto `PBNA`, que hace referencia al **Precio Bruto Nacional**

In [70]:
df_precios.UpLoadId.unique()

array([nan])

### Cambios a aplicar:

- Filtrado de DataFrame: `ConceptId`=`PBNA`
-Filtrado de columnas: Se identifica que las únicas columnas relevantes son `Date`, `Period`, `ConceptId` (solo para hacer el filtro, luego deja de ser relevante) y `value`

In [71]:
columns_to_keep=["Date","Period","ConceptId","Value"]

#Filtrando dataframe
df_precios_curated=df_precios[columns_to_keep] #filtrando filas
print("Sin filtrar filas ",df_precios_curated.shape)
df_precios_curated=df_precios_curated[df_precios_curated["ConceptId"]=="PBNA"] #filtrando columnas
print("Filtrando por conceptID ", df_precios_curated.shape)
df_precios_curated.head()

Sin filtrar filas  (21264, 4)
Filtrando por conceptID  (19340, 4)


,Date,Period,ConceptId,Value
0,2023-09-30 00:00:00.000,20,PBNA,1062.184761
1,2023-09-30 00:00:00.000,19,PBNA,1062.185761
2,2023-09-30 00:00:00.000,8,PBNA,1008.944761
3,2023-09-30 00:00:00.000,2,PBNA,1008.945761
4,2023-09-30 00:00:00.000,11,PBNA,1062.085761


### Formateando las columnas Date y Period.

La columna Date hace referencia a la fecha y la columna Period, hace referencia a un periodo de minutos del día, se van a realizar las transformaciones necesarias para concervir el Period en la hora del día.

In [72]:
print(len(df_precios.sort_values(by="Period",ascending=True).Period.unique()))
df_precios.sort_values(by="Period",ascending=True).Period.unique()

25


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

Cada `Period` hace referencia a una hora del día.

In [73]:
# Define a function to convert periods to hours and minutes
def period_to_time(period):
    if period == 0:
        return "00:00"
    elif period == 24:
        return "00:00"
    else:
        total_minutes = (period) * 60
        hours = total_minutes // 60
        minutes = total_minutes % 60
        return f"{hours:02d}:{minutes:02d}"

In [74]:

df_precios_curated["Date"] = df_precios_curated["Date"].apply(lambda x: x.split(" ")[0])
df_precios_curated["Hour"] = df_precios["Period"].apply(period_to_time)
df_precios_curated["Datetime"] = (
    df_precios_curated["Date"] + " " + df_precios_curated["Hour"]
)
df_precios_curated["Datetime"] = pd.to_datetime(df_precios_curated["Datetime"])
df_precios_curated.head()

,Date,Period,ConceptId,Value,Hour,Datetime
0,2023-09-30,20,PBNA,1062.184761,20:00,2023-09-30 20:00:00
1,2023-09-30,19,PBNA,1062.185761,19:00,2023-09-30 19:00:00
2,2023-09-30,8,PBNA,1008.944761,08:00,2023-09-30 08:00:00
3,2023-09-30,2,PBNA,1008.945761,02:00,2023-09-30 02:00:00
4,2023-09-30,11,PBNA,1062.085761,11:00,2023-09-30 11:00:00


### Conservando columnas relevantes

Después de filtrar las filas y formatear la fecha y hora, las columnas `Period` y `ConceptId` dejan de ser relevantes.

In [75]:
df_precios_curated=df_precios_curated.drop(columns=(["Period","ConceptId"]))
df_precios_curated.head()

,Date,Value,Hour,Datetime
0,2023-09-30,1062.184761,20:00,2023-09-30 20:00:00
1,2023-09-30,1062.185761,19:00,2023-09-30 19:00:00
2,2023-09-30,1008.944761,08:00,2023-09-30 08:00:00
3,2023-09-30,1008.945761,02:00,2023-09-30 02:00:00
4,2023-09-30,1062.085761,11:00,2023-09-30 11:00:00


## Analizando la Calidad de los datos.

### Análisis de Tipos de Datos

In [76]:
df_precios_curated.dtypes

Date                object
Value              float64
Hour                object
Datetime    datetime64[ns]
dtype: object

In [77]:
df_precios_curated["Date"]=pd.to_datetime(df_precios_curated["Date"])
print(df_precios_curated.dtypes)
df_precios_curated.head()

Date        datetime64[ns]
Value              float64
Hour                object
Datetime    datetime64[ns]
dtype: object


,Date,Value,Hour,Datetime
0,2023-09-30,1062.184761,20:00,2023-09-30 20:00:00
1,2023-09-30,1062.185761,19:00,2023-09-30 19:00:00
2,2023-09-30,1008.944761,08:00,2023-09-30 08:00:00
3,2023-09-30,1008.945761,02:00,2023-09-30 02:00:00
4,2023-09-30,1062.085761,11:00,2023-09-30 11:00:00


In [85]:
df_precios_curated["Year"]=df_precios_curated["Date"].dt.year
print(df_precios_curated.dtypes)
df_precios_curated.head()

Date        datetime64[ns]
Value              float64
Hour                object
Datetime    datetime64[ns]
Year                 int32
dtype: object


,Date,Value,Hour,Datetime,Year
0,2023-09-30,1062.184761,20:00,2023-09-30 20:00:00,2023
1,2023-09-30,1062.185761,19:00,2023-09-30 19:00:00,2023
2,2023-09-30,1008.944761,08:00,2023-09-30 08:00:00,2023
3,2023-09-30,1008.945761,02:00,2023-09-30 02:00:00,2023
4,2023-09-30,1062.085761,11:00,2023-09-30 11:00:00,2023


### Buscando Anomalías en `Value` o en `Date`

#### Gráficando distribución de `Value`

In [88]:
px.box(df_precios_curated,x="Value",title="Distribución del Precio de la Energia Kb/h")

No se identifican Valores Negativos o Atipicos en la columna `value`

#### Gráficando distribución de `Date`

In [90]:
px.box(df_precios_curated,x="Date",title="Distribución de Date")

#### Gráficando distribución de `DateTime`

In [92]:
px.box(df_precios_curated,x="Datetime",title="Distribución de Datetime")

No se identifican fechas incoherentes. Queda claro que hay datos desde el **01-01-2021** hasta **30-09-2023**

## Agregando datos y Realizando Analisis Exploratorio de Datos

Dado que los datos exógenos están a nivel de día, se van a agrupar los valores de precios de la energía para tener solo un dato por día, para esto se va a calcular el promedio por Fecha.

In [105]:
df_precio_dia=df_precios_curated.groupby(["Year","Date"])[["Value"]].mean().reset_index()
df_precio_dia

,Year,Date,Value
0,2021,2021-10-01,216.929726
1,2021,2021-10-02,256.940174
2,2021,2021-10-03,282.065065
3,2021,2021-10-04,286.526116
4,2021,2021-10-05,278.157325
...,...,...,...
639,2023,2023-09-26,1061.527174
640,2023,2023-09-27,1041.160576
641,2023,2023-09-28,1042.769645
642,2023,2023-09-29,1030.807224


### Analizando la tendencia del Precio de la energía Diario

In [111]:
px.box(df_precio_dia,y="Value",x="Year",title="Distribución del Precio de la Energia Kb/h por Year",color="Year")

In [113]:
px.line(df_precio_dia,y="Value",x="Date",title="Evolución precio de la energía")

In [114]:
df_precio_dia.to_csv("processed_tables/precios.csv",index=False)